# CNN

卷积神经网络CNN的结构一般包含这几个层： 
**输入层**：用于数据的输入。
**卷积层**：使用卷积核进行特征提取和特征映射。
**激励层**：由于卷积也是一种线性运算，因此需要增加非线性映射。
**池化层**：进行下采样，对特征图稀疏处理，减少数据运算量。 
**全连接层**：通常在CNN的尾部进行重新拟合，减少特征信息的损失。
**输出层**：用于输出结果。

## architecture

![CNN.png](https://img-blog.csdn.net/20180211155818513?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvbTBfMzczMDYzNjA=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms
from torch.autograd import Variable

# training settings
batch_size = 64 

# MNSIT dataset
train_dataset = datasets.MNIST(root='./data/', 
                               train=True,
                               transform=transforms.ToTensor(),
                               download=False
                              )

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor()
                             )

# data loader (input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True
                                          )

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batch_size,
                                           shuffle=True
                                          )

In [9]:
# CNN Net
class CNNNet(nn.Module):
    def __init__(self):
        super(CNNNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)
        
    def forward(self, x):
        # in_size = 64
        in_size = x.size(0) # one batch
        # x:64*10*12*12
        x = F.relu(self.mp(self.conv1(x)))
        # x:64*20*4*4
        x = F.relu(self.mp(self.conv2(x)))
        # x:64*3200
        x = x.view(in_size, -1) # flatten the tensor
        # x:64*10
        x = self.fc(x)
        return F.log_softmax(x)

In [10]:
model = CNNNet()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [16]:
# train
def train(epoch):
    for batch_idx,(data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 200 == 0:
            print('Train epoch:{} {}/{}({:.0f}%)\tLoss:{:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()
            ))

In [21]:
# test
def test():
    test_loss = 0
    correct = 0 
    for data, target in test_loader:
        with torch.no_grad():
            data, target = Variable(data), Variable(target)
            output = model(data)
            # sum up batch loss
            test_loss += F.nll_loss(output, target, size_average=False).item()
            # get the index of the max log-probability
            pred = output.data.max(1)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [23]:
print(model)
for epoch in range(1, 10):
    train(epoch)
    test()

CNNNet(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (mp): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=320, out_features=10, bias=True)
)
Train epoch:1 0/60000(0%)	Loss:0.022435


/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train epoch:1 12800/60000(21%)	Loss:0.048499
Train epoch:1 25600/60000(43%)	Loss:0.002146
Train epoch:1 38400/60000(64%)	Loss:0.018301
Train epoch:1 51200/60000(85%)	Loss:0.008258


/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.0405, Accuracy: 9863/10000 (98%)

Train epoch:2 0/60000(0%)	Loss:0.005861
Train epoch:2 12800/60000(21%)	Loss:0.024455
Train epoch:2 25600/60000(43%)	Loss:0.016191
Train epoch:2 38400/60000(64%)	Loss:0.060093
Train epoch:2 51200/60000(85%)	Loss:0.239747

Test set: Average loss: 0.0369, Accuracy: 9884/10000 (98%)

Train epoch:3 0/60000(0%)	Loss:0.017807
Train epoch:3 12800/60000(21%)	Loss:0.003735
Train epoch:3 25600/60000(43%)	Loss:0.000745
Train epoch:3 38400/60000(64%)	Loss:0.022618
Train epoch:3 51200/60000(85%)	Loss:0.019368

Test set: Average loss: 0.0381, Accuracy: 9865/10000 (98%)

Train epoch:4 0/60000(0%)	Loss:0.004647
Train epoch:4 12800/60000(21%)	Loss:0.048421
Train epoch:4 25600/60000(43%)	Loss:0.036511
Train epoch:4 38400/60000(64%)	Loss:0.004348
Train epoch:4 51200/60000(85%)	Loss:0.003322

Test set: Average loss: 0.0389, Accuracy: 9870/10000 (98%)

Train epoch:5 0/60000(0%)	Loss:0.003027
Train epoch:5 12800/60000(21%)	Loss:0.007592
Train epoch